In [ ]:
#@title 環境構築
#remove " > /dev/null 2>&1" ログの非表示
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[all] > /dev/null 2>&1
!git clone  https://github.com/GaetanJUVIN/Deep_QLearning_CartPole.git 2>&1
!cp /content/Deep_QLearning_CartPole/cartpole_weight.h5 /content/ 2>&1
!apt-get install xvfb > /dev/null 2>&1
# ライブラリのインポート
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

# openAi Gym のレンダリング結果をmp4として保存し表示する機能
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                 controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

     |████████████████████████████████| 788kB 2.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: setuptools 49.6.0
    Uninstalling setuptools-49.6.0:
      Successfully uninstalled setuptools-49.6.0


Cloning into 'Deep_QLearning_CartPole'...
remote: Enumerating objects: 15, done.
remote: Total 15 (delta 0), reused 0 (delta 0), pack-reused 15
Unpacking objects: 100% (15/15), done.


#HAIM
9月5日<br>
<p><font size="7" color="red">強化学習入門</font></p>

#自己紹介
名前　山本大悟  
仕事　富山県でsier  
興味　機械学習  

# このLTの流れ

1. 強化学習の理論を簡単に説明
2. openAI Gymを紹介
3. openAI Gymを用いて強化学習のコードを動かす。


# 強化学習とは何か
- 強化学習は最適な意思決定のルールを求めることを目的とする学問分野である。
- 利用されている分野はゲームのAI(将棋、囲碁)、ピッキングロボ(商品を取り出して移動をするロボット)


# 用語説明
- エージェント：環境に対して行動を起こす学習者。強化学習では、このエージェントが環境に対して様々な試行を繰り返すことで行動を最適化していきます。
- 環境：エージェントの行動に対して状態の更新と報酬の付与を行います。
- 強化学習では、与えられた「環境」における価値を最大化するように「エージェント」を学習させます。
- 状態(s_t)：環境が保持する環境の様子で、エージェントが起こす行動によって変化します。
- 行動(a)：エージェントが、ある状態()のときに取ることができる行動のことです。
- 報酬(r)：エージェントの行動に対する環境からの報酬です。

# 仕組みは?
- 強化学習はエージェントと環境の相互作用から学習を行う。
- 将来にわたって得られる報酬の和を最大化することを目的としている。　

<img src="https://gym.openai.com/assets/docs/aeloop-138c89d44114492fd02822303e6b4b07213010bb14ca5856d2d49d6b62d88e53.svg" width=450px>




# OpenAIとGymについて

# OpenAIとは[1]？
- OpenAIはAIの研究を行う非営利団体である。AIの研究成果を自己利益のためではなく、人類全体のために活用することを目的としている。
- OpenAIの研究成果は公開されている。主な成果物には文章を自動で生成する「GPT2」や画像を自動生成する「Image GPT」、後ほど紹介する「Gym」がある。

## Image GPTでの画像生成例
![](https://i.imgur.com/Y8tywQ6.png)

## Gym とは[2][3]？
- 強化学習のAIを開発評価するため環境を提供するプラットフォーム
- Atari社のゲームのパックンマンやブロック崩し、スペースインベーダーなどのAIを開発できる。

![](https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F396969%2Fce95c737-755e-7174-b246-d3ea4a548a61.gif?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=c25dc0f1f33e4d46583b3941aed8b9ef)
<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F3180%2F8c235a00-cd55-41a2-a605-a4a2e9b0240f.gif?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&w=1400&fit=max&s=d76ac9841e0127bfde116d347587820a" width=150px>


## 参考資料
[1]https://openai.com/projects/  
[2]https://qiita.com/shu223/items/a9cfe9107447b327b564  
[3]https://gym.openai.com/envs/#atari

In [ ]:
#@title Gymを動かしてみる！
NAME = "CartPole-v0"#@param["MsPacman-v0","Breakout-v0","CartPole-v0","CarRacing-v0"]
env = wrap_env(gym.make(NAME))
observation = env.reset()
while True:
    env.render()
    action = env.action_space.sample() 
    observation, reward, done, info = env.step(action) 
    if done: 
      break;
env.close()
show_video()

# CartPoleを題材に強化学習を学ぶ

## CartPole(倒立振り子課題)とは？
CartPoleとは土台の上に回転軸を固定した棒を動かして、立てた棒が倒れないように土台を右、左に細かく動かして制御する課題である。イメージとしては、小学生のころ掃除の時間に手のひらの上にほうきを立てる遊びと似ている。[1]
![](https://book.mynavi.jp/files/user/manatee/rensai/019_kyoukagakusyuu/09/cartpole.gif)

## GymのCartPoleのルールは？
### 開始状態
±5度以下に傾いた状態からスタートする
### CartPoleの終了条件
- ポールの角度が±12度以上傾く。
- 土台が画面外に出る
- 一定の時間(10秒)を超える。
### クリア条件
- 10秒以上倒れないこと

### 得られる情報　
- 台車の速度やポールの角度の情報
- 報酬　倒れていないと１　倒れると０

### 行える動作
 - 台車を右に押すか左に押すか

## 参考資料
[1]https://book.mynavi.jp/manatee/detail/id=88961#:~:text=%E5%80%92%E7%AB%8B%E6%8C%AF%E5%AD%90%E8%AA%B2%E9%A1%8CCartPole%E3%81%A8%E3%81%AF&text=%E5%80%92%E7%AB%8B%E6%8C%AF%E5%AD%90%E8%AA%B2%E9%A1%8C%E3%81%A8%E3%81%AF,%E3%81%BE%E3%81%A3%E3%81%9F%E3%81%8F%E5%90%8C%E3%81%98%E3%81%AB%E3%81%AA%E3%82%8A%E3%81%BE%E3%81%99%E3%80%82




#PID制御を用いる

![](https://image.itmedia.co.jp/mn/articles/1005/21/ay_etr03_siki01.gif)

OpenAI GymのCartPoleの場合  
それぞれの係数は
0.1、0.01、0.5としている。

In [ ]:
#@title  CartPoleを PID制御で解く
import numpy as np
import gym

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

env = wrap_env(gym.make('CartPole-v0'))
desired_state = np.array([0, 0, 0, 0])
desired_mask = np.array([0, 0, 1, 0])

P, I, D = 0.1, 0.01, 0.5

for i_episode in range(1):
    state = env.reset()
    integral = 0
    derivative = 0
    prev_error = 0
    for t in range(500):
        env.render()
        error = state - desired_state

        integral += error
        derivative = error - prev_error
        prev_error = error

        pid = np.dot(P * error + I * integral + D * derivative, desired_mask)
        action = sigmoid(pid)
        action = np.round(action).astype(np.int32)

        state, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()
show_video()

Episode finished after 200 timesteps


# 強化学習のDeep Q Networkを使って解く

1. ε-greedy法  
確率εに従い探索(exploration)と活用(exploitation)を決める方法
- 探索はランダムに行動を選択する
- 活用は評価値が最大になる行動を選択する (greedy法) 

greedy法のみを用いると幅広い行動を行うことができず学習が進まなくなるため、ランダムに行動させる。

2.Deep Q Network
  1. Kerasでモデルを作成する
  2. 状態を入力データとして評価値を教師データとしてモデルの学習を進める


![](https://i.imgur.com/SIvXJdL.png)



![](https://i.imgur.com/npSDWYJ.png)

In [ ]:
#@title DQNの学習
import gym
import random
import os
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam


class Agent():
    def __init__(self, state_size, action_size):
        self.weight_backup = "cartpole_weight.h5"
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.learning_rate = 0.001
        self.gamma = 0.95
        self.exploration_rate = 1.0
        self.exploration_min = 0.01
        self.exploration_decay = 0.995
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))

        if os.path.isfile(self.weight_backup):
            model.load_weights(self.weight_backup)
            self.exploration_rate = self.exploration_min
        return model

    def save_model(self):
        self.model.save(self.weight_backup)

    def act(self, state):
        if np.random.rand() <= self.exploration_rate:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, sample_batch_size):
        if len(self.memory) < sample_batch_size:
            return
        sample_batch = random.sample(self.memory, sample_batch_size)
        for state, action, reward, next_state, done in sample_batch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.exploration_rate > self.exploration_min:
            self.exploration_rate *= self.exploration_decay
class CartPole:
    def __init__(self):
        self.sample_batch_size = 32
        self.episodes = 1
        self.env = wrap_env(gym.make('CartPole-v0'))

        self.state_size = self.env.observation_space.shape[0]
        self.action_size = self.env.action_space.n
        self.agent = Agent(self.state_size, self.action_size)

    def run(self):
        try:
            for index_episode in range(self.episodes):
                state = self.env.reset()
                state = np.reshape(state, [1, self.state_size])

                done = False
                index = 0
                while not done:
                    #self.env.render()

                    action = self.agent.act(state)

                    next_state, reward, done, _ = self.env.step(action)
                    next_state = np.reshape(next_state, [1, self.state_size])
                    self.agent.remember(state, action, reward, next_state, done)
                    state = next_state
                    index += 1
                print("Episode {}# Score: {}".format(index_episode, index + 1))
                self.agent.replay(self.sample_batch_size)
        finally:
            self.agent.save_model()
cartpole = CartPole()
cartpole.run()


In [ ]:
#@title 学習済みモデルを用いてCartPoleを動かしてみる！
NAME = "CartPole-v0"
env = wrap_env(gym.make(NAME))
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
state = env.reset()
state = np.reshape(state, [1, state_size])
agent = Agent(state_size, action_size)
while True:
    env.render()
    action = agent.act(state)
    observation, reward, done, info = env.step(action)
    observation = np.reshape(observation, [1, state_size]) 
    state = observation
    if done: 
      break;
env.close()
show_video()